# GEOSPACIAL VISUALIZATION PROJECT

- Mi empresa:
    - Compañia de software con 50 empleados.
    - Producto: videojuegos
    - Empleados: 20 developers, 20 Designers/Creatieves/UX/UI and 10 executives/managers
     
- Requisitos buscados para localizar la empresa: radio de cercania: 50 km

        --> offices.latitude
        --> offices.longitude
        
    - Debe haber software engineers trabajando cerca:
        - category_code
        
    - Debe haber un buen ratio entre grandes compañias / startups
        - number_of_employees: 0-10 startup; 11-100 small; 100-999 medium; >1000 big
        - total_money_raised
        
    - Empresas cercanas deben cubrir los intereses de mis trabajadores --> Text mining
        - tag_list
        - twitter_username: nº de seguidores, nºtweets
        
    - Prefiero estar cerca de compañias nuevas que viejas
        - deadpooled_year: null -- ELimino las compañias que quebraron
        - founded_year: considero solo compañias con 10 años de antiguedad maximo

In [1]:
import pandas as pd
import numpy as np
import re
import pymongo

## 1. Limpiar el data set para seleccionar solo las compañias que me interesan.

In [2]:
# Cargar la base de datos de Mongodb

MongoClient = pymongo.MongoClient
client = MongoClient()
db = client.companies

In [3]:
# Crear un cursor con los campos que me interesan y filtrando de entrada ciertas empresas. 

companies_interest = list(db.companies.find({"deadpooled_year": None},
                                       {"_id":0,"name":1,"category_code":1,"number_of_employees":1,
                                       "total_money_raised":1,"tag_list":1,"founded_year":1,"twitter_username":1,
                                       "offices.latitude":1,"offices.longitude":1}))

In [4]:
# Crear un dataframe a partir del cursor extraido de mongodb

df_companies = pd.DataFrame(companies_interest)

display(df_companies.shape)

(17872, 8)

In [5]:
# Extraer de la columna offices las coordenadas para pasarlas a GeoJson

df_= pd.DataFrame(pd.DataFrame(df_companies["offices"].values.flatten([0]).tolist()))
df_companies = pd.concat([df_,df_companies],axis=1)

/usr/lib/python3/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Non-string object detected for the array ordering. Please pass in 'C', 'F', 'A', or 'K' instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
# Al hacer esta acción se crean numerosas columnas (66), me quedo solo con la primera.

df_companies = df_companies[['name','category_code',0,'founded_year','number_of_employees',
                             'total_money_raised','tag_list','twitter_username']].copy()

df_companies.columns= ['name','category','coords','founded_year','employees','total_money_raised','tags','twitter']

In [7]:
df_companies.head()

,name,category,coords,founded_year,employees,total_money_raised,tags,twitter
0,Digg,news,"{'latitude': 37.764726, 'longitude': -122.394523}",2004.0,60.0,$45M,"community, social, news, bookmark, digg, techn...",digg
1,Flektor,games_video,"{'latitude': 34.025958, 'longitude': -118.379768}",NaN,NaN,$0,"flektor, photo, video",None
2,Postini,web,"{'latitude': 37.506885, 'longitude': -122.247573}",1999.0,NaN,$0,,None
3,Geni,web,"{'latitude': 34.090368, 'longitude': -118.393064}",2006.0,18.0,$16.5M,"geni, geneology, social, family, genealogy",geni
4,Fox Interactive Media,web,"{'latitude': 34.076179, 'longitude': -118.39417}",1979.0,0.0,$0,,twitterapi


In [8]:
# Columnas 'founded_year' y 'employees': Eliminar nulos y cambiar tipo de dato. Filtro empresas > 10 años

df_companies = df_companies[np.isfinite(df_companies['founded_year'])]
df_companies = df_companies[np.isfinite(df_companies['employees'])]

df_companies['employees'] = df_companies['employees'].astype('int64')
df_companies['founded_year'] = df_companies['founded_year'].astype('int64')

df_companies = df_companies.loc[df_companies["founded_year"]>=2008]

In [9]:
# Creo bins con 'employees': startup, pequeña, mediana y gran empresa

bins = [0,10,100,999,500000]
labels = ["startup","small","medium","big"]
df_companies['tamaño_empresa'] = pd.cut(df_companies['employees'], bins=bins, labels=labels)

df_companies.dropna(subset=['tamaño_empresa'], inplace=True)

df_companies.groupby(df_companies.tamaño_empresa)['name'].nunique()  # mirar al final

tamaño_empresa
startup    1728
small       443
medium       45
big           5
Name: name, dtype: int64

In [10]:
# En 'total_money_raised' extraigo el valor en millones de $ como indicador de tamaño de empresa. Elimino 0.0M

def change_row (row):
    row = ''.join((ch if ch in '0123456789.' else ' ') for ch in row)
    return row

df_companies['total_money_raised'] = (df_companies['total_money_raised'].apply(change_row))

df_companies['total_money_raised(M)'] = df_companies['total_money_raised'].astype('float64')

df_companies = df_companies.loc[df_companies['total_money_raised(M)']!=0.0]
df_companies = df_companies.loc[df_companies['total_money_raised(M)']!=0.00]


In [11]:
# Columna 'coords': elimino los nulos y la paso a formato GeoJson

df_companies.dropna(subset=['coords'], inplace=True)

def toGeoJSON(array):
    return {
        "type":"Point",
        "coordinates":[array['longitude'],array['latitude']]
    }

df_companies['coords'] = df_companies.apply(lambda r: toGeoJSON(r['coords']), axis=1)

df_companies = df_companies.loc[df_companies['coords']!= {'type': 'Point', 'coordinates': [None, None]}]

In [12]:
# Columna 'category': filtro por las que me interesa tener cerca

df_companies.dropna(subset=['category'], inplace=True)

df_companies = df_companies.loc[df_companies['category'].
                                isin(['web','games_video','mobile','network_hosting','hardware','software',
                                      'ecommerce','public_relations','finance','analytics','sports','design'])]

In [13]:
df_companies = df_companies.reset_index()
df_companies = df_companies.drop(['index','total_money_raised'], axis=1)

In [14]:
# Guardo el archivo limpio como json (GeoJson)

df_companies.to_json("CleanCompanies.json", orient="records", lines=True)

## 2. Cargar el nuevo data set en MongoDb para poder hacer las preguntas geo.

In [15]:
# Creo un diccionario con key:name, value:coordenadas

list_coord = list(df_companies['coords'])
list_coord = list (map(lambda d: d['coordinates'], list_coord))

dict_coord = dict(zip(list(df_companies['name']),list_coord))

In [16]:
# Creo una función que me devuelva las empresas en un radio de 5 km de un punto concreto en el mapa.

def near_companies(coords): 
    cursor = db.clean_companies.find({
      "coords": {
        "$near": {
          "$geometry": {
            "type": "Point",
            "coordinates": [coords[0], coords[1]]
          },
          "$minDistance": 0,
          "$maxDistance":5000
        }
      }
    })
    
    df_geocomp = pd.DataFrame(list(cursor))
    
    return df_geocomp

# Esta devuelve no solo el número de empresas, también su categoria y su tamaño

def results_df (df_geocomp):
    results_df = df_geocomp[['name','category','tamaño_empresa']].copy()
    return results_df

# La siguiente devuelve el número de empresas cercanas

def number (df_geocomp):
    return df_geocomp['name'].count()

In [17]:
# Ver el número de empresas que tiene cada una de las empresas cerca

for name, value in dict_coord.items():
    print (name, number(near_companies(value)))

GoingOn 62
Crunchyroll 62
GENWI 1
Fixya 9
First30Days 1
Social Gaming Network 27
Stumpedia 1
99designs 62
Skydeck 9
Magento 2
Intent 11
Noovo 27
AudioMicro 1
Mojiva 50
GitHub 64
Mad Mimi 3
UserVoice 1
Serious Business 62
New Relic 3
Jinni 4
Smarkets 14
Stitcher 64
Socruise 1
Pinch Media 53
Orbster 2
BrandYourself 2
Tapulous 27
Titan Gaming 3
gAuto 3
Posterous 64
Open Places 58
TweetDeck 23
Wahanda 1
IBTgames 1
Samfind 11
Prezi 2
ngmoco 62
CompareAway 1
Integrate 43
vufind 25
Poll Everywhere 13
YadaHome 8
Occipital 5
iCharts 9
Producteev 39
Twonq 63
Tweegee 4
Spare Change Payments 4
Lolapps 62
InnerRewards 63
DealBase Corporation 8
Cliqset 1
Groupon 14
BuzzStream 10
Big In Japan 5
iKlax Media 1
Sailthru 53
AlphaClone 63
Formspring 1
Wishabi 4
CarZen 36
Dukky 1
BrightScope 4
Yipit 56
FiPath 6
Virgance 62
Saltlick Labs 4
Double Encore 1
Movirtu 14
mobiManage 1
People Capital 53
MarketRiders 1
Silver Tail Systems 26
Tengaged 6
Drync 3
Coveroo 62
ZimpleMoney 2
Wealthfront 27
StockTwits 2
RJ

In [19]:
# Me voy a quedar inicialmente con las que tienen mas de 50 empresas de las seleccionadas a su alrededor

list_companies= []
for name, value in dict_coord.items():
    if int(number(near_companies(value)))>=50:
        list_companies.append(name)

# Creo un diccionario con estas empresas para representarlas en el mapa

CompaniesSelect_1 = {k: dict_coord[k] for k in dict_coord.keys() & {'GoingOn', 'Crunchyroll', '99designs', 'Mojiva',
                                                                    'GitHub', 'Serious Business', 'Stitcher',
                                                                    'Pinch Media', 'Posterous', 'Open Places',
                                                                    'ngmoco', 'Twonq', 'Lolapps', 'InnerRewards',
                                                                    'Sailthru', 'AlphaClone', 'Yipit', 'Virgance',
                                                                    'People Capital', 'Coveroo', '5th Finger',
                                                                    'Meez', 'KODA', 'Wee Web', 'Kapitall',
                                                                    'Home-Account', 'IntheGlo', 'Chartbeat',
                                                                    'Udorse', 'ChallengePost', 'StyleCaster'}}

CompaniesSelect_1 = pd.DataFrame.from_dict(CompaniesSelect_1, orient='index')

CompaniesSelect_1 = CompaniesSelect_1.reset_index()

CompaniesSelect_1.columns = ['name','long','lat']

# Creo un json

CompaniesSelect_1.to_json("CompaniesSelect_1.json",orient='records')

## 3. Creo un data set para representar todas las empresas de la primera limpieza

In [20]:
All_Companies = df_companies[['name','category','coords','tamaño_empresa']]

df_= pd.DataFrame(pd.DataFrame(All_Companies["coords"].values.flatten([0]).tolist()))
All_Companies = pd.concat([df_,All_Companies],axis=1)


coord = All_Companies['coordinates'].apply(pd.Series)
All_Companies = pd.concat([All_Companies[:], coord[:]], axis=1)
All_Companies = All_Companies.drop(['coords','type','coordinates'], axis=1)

All_Companies.columns = ['name','category','tamaño_empresa','long','lat']

All_Companies.head()

/usr/lib/python3/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Non-string object detected for the array ordering. Please pass in 'C', 'F', 'A', or 'K' instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,category,tamaño_empresa,long,lat
0,GoingOn,software,small,-122.392142,37.782263
1,Crunchyroll,games_video,small,-122.393229,37.781265
2,GENWI,mobile,small,-111.903500,33.817100
3,Fixya,web,small,-122.323895,37.566879
4,First30Days,web,small,-85.717393,38.257035


In [21]:
# Creo un json

All_Companies.to_json("All_Companies.json",orient='records')

### Finalmente los puntos elegidos son:
    - Principalmente la zona de Market Streeten San Francisco (US A): (37.787,-122.40)
    - Manhattan la zona de la 14th con la 4th, Nueva York (USA): (40.733,-73.99)
    
* He utilizado varios mapas ya que no  he podido comparar mediante dataframes los datos de categoria y tamaño de las empresas. Por ello, tras ver el mapa con las empresas con mas empresas cercanas dentro del filtro (que incluian empresas de todo tamaño y algunas categorias concretas), he observado un mapa con todas las empresas tras la primera limpieza, y ahí he podido ver con detalle el tamaño de las empresas de la zona y tambien la categoria. 